# Day8_1: 시계열 분석 기법 (resample, rolling, 분해)

## 📚 학습 목표

**Part 1: 기초**
1. resample()로 시계열 데이터 리샘플링하기
2. 업샘플링과 다운샘플링의 차이 이해하기
3. 다양한 집계 함수로 주기 변환하기
4. rolling()으로 이동 평균 계산하기
5. expanding()으로 누적 통계 구하기

**Part 2: 심화**
1. 지수 가중 이동 평균(EWM) 활용하기
2. 시계열 분해로 추세와 계절성 추출하기
3. Plotly로 시계열 분해 결과 시각화하기
4. 이동 통계를 활용한 이상치 탐지하기

---

## 🎯 왜 이것을 배우나요?

| 개념 | 실무 활용 | 예시 |
|------|----------|------|
| resample | 시간 단위 통합/확장 | 일별 매출 → 월별 매출, 시간별 → 분별 |
| rolling | 노이즈 제거, 추세 파악 | 주가 이동평균선, 7일 평균 매출 |
| expanding | 누적 성과 추적 | 연초부터 현재까지 누적 매출 |
| 시계열 분해 | 패턴 분석, 예측 모델링 | 계절성 제거 후 추세 예측 |

**분석가 관점**: 주식 차트의 이동평균선, 쇼핑몰 계절성 분석, 웹 트래픽 추세 파악 등 시계열 패턴을 발견하고 노이즈를 제거하는 필수 기법입니다!

---

In [ ]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta

# 한글 폰트 설정 (Plotly)
import plotly.io as pio
pio.templates.default = "plotly_white"

print("✅ 라이브러리 로드 완료")

# Part 1: 기초

---

## 1.1 resample()로 리샘플링하기

### resample()이란?

- **시계열 데이터의 주기를 변경**하는 메서드
- **다운샘플링**: 고빈도 → 저빈도 (예: 일별 → 월별)
- **업샘플링**: 저빈도 → 고빈도 (예: 월별 → 일별)
- DatetimeIndex가 필요함

In [ ]:
# 일별 매출 데이터 생성
np.random.seed(42)
dates = pd.date_range(start='2024-01-01', end='2024-01-31', freq='D')
daily_sales = pd.DataFrame({
    'date': dates,
    'sales': np.random.randint(100, 300, size=len(dates))
})

# DatetimeIndex로 설정
daily_sales.set_index('date', inplace=True)
print("일별 매출 데이터 (처음 5개):")
print(daily_sales.head())
print(f"\n총 {len(daily_sales)}개 레코드")

### 다운샘플링: 일별 → 주별

In [ ]:
# 주별(Weekly) 집계 - 합계
weekly_sales_sum = daily_sales.resample('W').sum()
print("주별 매출 합계:")
print(weekly_sales_sum)

# 주별 평균
weekly_sales_mean = daily_sales.resample('W').mean()
print("\n주별 매출 평균:")
print(weekly_sales_mean)

### 다양한 주기 코드

| 코드 | 의미 | 예시 |
|------|------|------|
| `'D'` | 일(Day) | 매일 |
| `'W'` | 주(Week) | 매주 일요일 |
| `'M'` | 월말(Month End) | 각 월 마지막 날 |
| `'MS'` | 월초(Month Start) | 각 월 첫날 |
| `'Q'` | 분기(Quarter) | 3개월 |
| `'Y'` | 연말(Year End) | 12월 31일 |
| `'H'` | 시간(Hour) | 매시간 |
| `'T'` or `'min'` | 분(Minute) | 매분 |

In [ ]:
# 월별 집계
monthly_sales = daily_sales.resample('M').agg({
    'sales': ['sum', 'mean', 'max', 'min', 'count']
})

print("월별 매출 집계:")
print(monthly_sales)

### 💡 실무 예시: 일별 주가 → 월별 요약

주식 데이터를 월별로 집계하여 OHLC (Open, High, Low, Close) 차트 생성

In [ ]:
# 주가 데이터 시뮬레이션 (종가)
np.random.seed(100)
dates_stock = pd.date_range('2024-01-01', '2024-12-31', freq='D')
stock_price = pd.DataFrame({
    'close': 50000 + np.cumsum(np.random.randn(len(dates_stock)) * 500)
}, index=dates_stock)

# 월별 OHLC 계산
monthly_ohlc = stock_price.resample('M').agg({
    'close': ['first', 'max', 'min', 'last']
})

monthly_ohlc.columns = ['Open', 'High', 'Low', 'Close']
print("월별 OHLC:")
print(monthly_ohlc.head())

---

## 1.2 업샘플링과 결측치 처리

### 업샘플링이란?

- 낮은 빈도 → 높은 빈도로 변환
- 새로 생긴 인덱스는 **결측치(NaN)**가 됨
- `.ffill()`, `.bfill()`, `.interpolate()`로 채우기

In [ ]:
# 월별 데이터 생성
monthly_data = pd.DataFrame({
    'revenue': [1000, 1200, 1100, 1300]
}, index=pd.date_range('2024-01', periods=4, freq='MS'))

print("월별 데이터:")
print(monthly_data)

In [ ]:
# 일별로 업샘플링 (결측치 발생)
daily_upsampled = monthly_data.resample('D').asfreq()
print("일별 업샘플링 (처음 10개):")
print(daily_upsampled.head(10))

In [ ]:
# Forward Fill (앞 값으로 채우기)
daily_ffill = monthly_data.resample('D').ffill()
print("Forward Fill (처음 35일):")
print(daily_ffill.head(35))

In [ ]:
# Interpolate (선형 보간)
daily_interpolated = monthly_data.resample('D').interpolate(method='linear')
print("선형 보간 (처음 35일):")
print(daily_interpolated.head(35))

### 💡 실무 예시: 주별 예산을 일별로 분배

마케팅 예산을 주별로 받았는데, 일별로 균등 분배하고 싶을 때

In [ ]:
# 주별 예산
weekly_budget = pd.DataFrame({
    'budget': [7000, 10500, 14000, 10500]  # 단위: 만원
}, index=pd.date_range('2024-01-07', periods=4, freq='W'))

print("주별 예산:")
print(weekly_budget)

# 일별로 보간 후 7로 나누기 (주당 7일)
daily_budget = weekly_budget.resample('D').interpolate() / 7
print("\n일별 예산 (처음 14일):")
print(daily_budget.head(14))

---

## 1.3 rolling()으로 이동 평균

### 이동 평균(Moving Average)이란?

- **일정 기간의 평균**을 계속 계산하며 이동
- 노이즈 제거, 추세 파악에 유용
- 주식 차트의 5일선, 20일선, 60일선 등

In [ ]:
# 일별 매출 데이터 (노이즈가 있는)
np.random.seed(50)
dates_ma = pd.date_range('2024-01-01', periods=60, freq='D')
sales_noisy = pd.DataFrame({
    'sales': 200 + np.cumsum(np.random.randn(60) * 10) + np.random.randn(60) * 20
}, index=dates_ma)

print("원본 매출 (처음 10개):")
print(sales_noisy.head(10))

In [ ]:
# 7일 이동 평균
sales_noisy['MA_7'] = sales_noisy['sales'].rolling(window=7).mean()

# 30일 이동 평균
sales_noisy['MA_30'] = sales_noisy['sales'].rolling(window=30).mean()

print("이동 평균 추가 (처음 35개):")
print(sales_noisy.head(35))

### rolling() 파라미터

- `window`: 윈도우 크기 (필수)
- `min_periods`: 최소 데이터 개수 (기본값: window와 동일)
- `center`: True면 중앙 정렬 (기본값: False)
- `.mean()`, `.sum()`, `.std()`, `.max()`, `.min()` 등 집계 함수

In [ ]:
# 이동 표준편차 (변동성 측정)
sales_noisy['rolling_std'] = sales_noisy['sales'].rolling(window=7).std()

print("이동 표준편차 (처음 15개):")
print(sales_noisy[['sales', 'rolling_std']].head(15))

### 💡 실무 예시: 주가 골든크로스 탐지

단기 이동평균선이 장기 이동평균선을 상향 돌파하면 매수 신호

In [ ]:
# 주가 시뮬레이션
np.random.seed(123)
stock_data = pd.DataFrame({
    'price': 50000 + np.cumsum(np.random.randn(100) * 500)
}, index=pd.date_range('2024-01-01', periods=100, freq='D'))

# 5일선, 20일선
stock_data['MA5'] = stock_data['price'].rolling(5).mean()
stock_data['MA20'] = stock_data['price'].rolling(20).mean()

# 골든크로스 탐지 (5일선 > 20일선 && 전날은 반대)
stock_data['golden_cross'] = (
    (stock_data['MA5'] > stock_data['MA20']) &
    (stock_data['MA5'].shift(1) <= stock_data['MA20'].shift(1))
)

print("골든크로스 발생일:")
print(stock_data[stock_data['golden_cross']])

---

## 1.4 expanding()으로 누적 통계

### expanding()이란?

- **처음부터 현재까지** 모든 데이터를 대상으로 계산
- 누적 평균, 누적 합계, 누적 최대값 등
- `window`가 계속 확장됨

In [ ]:
# 일별 매출
daily_revenue = pd.DataFrame({
    'revenue': [100, 120, 90, 150, 130, 140, 110, 160, 180, 170]
}, index=pd.date_range('2024-01-01', periods=10, freq='D'))

# 누적 합계 (연초부터 현재까지)
daily_revenue['cumulative_sum'] = daily_revenue['revenue'].expanding().sum()

# 누적 평균
daily_revenue['cumulative_mean'] = daily_revenue['revenue'].expanding().mean()

print("누적 통계:")
print(daily_revenue)

### 💡 실무 예시: 연초부터 누적 매출 목표 달성률

In [ ]:
# 일별 목표 매출
target_daily = 150  # 만원

# 누적 매출 vs 누적 목표
daily_revenue['cumulative_target'] = target_daily * (daily_revenue.index - daily_revenue.index[0]).days + target_daily
daily_revenue['achievement_rate'] = (daily_revenue['cumulative_sum'] / daily_revenue['cumulative_target'] * 100).round(2)

print("누적 매출 목표 달성률:")
print(daily_revenue[['revenue', 'cumulative_sum', 'cumulative_target', 'achievement_rate']])

---

# Part 2: 심화

---

## 2.1 지수 가중 이동 평균 (EWM)

### EWM (Exponentially Weighted Moving Average)

- **최근 데이터에 더 큰 가중치**를 부여
- 단순 이동평균보다 빠르게 최근 변화를 반영
- 금융 시계열 분석에서 널리 사용
- `span` 파라미터로 가중치 조절 (클수록 평활화 효과 증가)

In [ ]:
# 주가 데이터
np.random.seed(200)
stock = pd.DataFrame({
    'price': 100 + np.cumsum(np.random.randn(50) * 2)
}, index=pd.date_range('2024-01-01', periods=50, freq='D'))

# 단순 이동평균 (SMA)
stock['SMA_10'] = stock['price'].rolling(10).mean()

# 지수 가중 이동평균 (EWM)
stock['EWM_10'] = stock['price'].ewm(span=10, adjust=False).mean()

print("SMA vs EWM (처음 15개):")
print(stock.head(15))

In [ ]:
# 시각화: SMA vs EWM 비교
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=stock.index, y=stock['price'],
    mode='lines', name='원본 가격',
    line=dict(color='lightgray', width=1)
))

fig.add_trace(go.Scatter(
    x=stock.index, y=stock['SMA_10'],
    mode='lines', name='SMA (10일)',
    line=dict(color='blue', width=2)
))

fig.add_trace(go.Scatter(
    x=stock.index, y=stock['EWM_10'],
    mode='lines', name='EWM (span=10)',
    line=dict(color='red', width=2, dash='dash')
))

fig.update_layout(
    title='단순 이동평균 vs 지수 가중 이동평균',
    xaxis_title='날짜',
    yaxis_title='가격',
    hovermode='x unified'
)

fig.show()

### 💡 실무 예시: 변동성(Volatility) 측정

주가의 표준편차를 EWM으로 계산하여 최근 변동성 추적

In [ ]:
# 일별 수익률
stock['daily_return'] = stock['price'].pct_change() * 100

# 변동성 (EWM 표준편차)
stock['volatility'] = stock['daily_return'].ewm(span=20, adjust=False).std()

print("변동성 (처음 25개):")
print(stock[['price', 'daily_return', 'volatility']].head(25))

---

## 2.2 시계열 분해 (Decomposition)

### 시계열 분해란?

시계열 데이터를 3가지 요소로 분리:

1. **Trend (추세)**: 장기적인 상승/하락 패턴
2. **Seasonality (계절성)**: 주기적으로 반복되는 패턴
3. **Residual (잔차)**: 설명되지 않는 랜덤 노이즈

**분해 모델**:
- **Additive (가산)**: `Y = Trend + Seasonality + Residual`
- **Multiplicative (승산)**: `Y = Trend × Seasonality × Residual`

In [ ]:
# statsmodels 설치 필요: uv add statsmodels
from statsmodels.tsa.seasonal import seasonal_decompose

# 계절성이 있는 데이터 생성 (2년치 월별)
np.random.seed(42)
dates_decomp = pd.date_range('2022-01', periods=24, freq='MS')

# 추세 (상승) + 계절성 (연말 피크) + 노이즈
trend = np.linspace(100, 200, 24)
seasonality = 20 * np.sin(np.linspace(0, 4*np.pi, 24))
noise = np.random.randn(24) * 5

sales_seasonal = pd.DataFrame({
    'sales': trend + seasonality + noise
}, index=dates_decomp)

print("계절성 데이터:")
print(sales_seasonal)

In [ ]:
# 시계열 분해 (Additive)
decomposition = seasonal_decompose(
    sales_seasonal['sales'],
    model='additive',
    period=12  # 12개월 주기
)

# 분해 결과
trend_component = decomposition.trend
seasonal_component = decomposition.seasonal
residual_component = decomposition.resid

print("추세 (Trend):")
print(trend_component.head(15))
print("\n계절성 (Seasonality):")
print(seasonal_component.head(15))
print("\n잔차 (Residual):")
print(residual_component.head(15))

### 💡 실무 예시: 소매업 계절성 분석

연말 쇼핑 시즌, 명절 등의 계절 효과를 제거하고 순수한 성장 추세 파악

In [ ]:
# 분해 결과를 DataFrame으로
decomp_df = pd.DataFrame({
    'original': sales_seasonal['sales'],
    'trend': trend_component,
    'seasonal': seasonal_component,
    'residual': residual_component
})

# 계절성 제거 데이터 (Deseasonalized)
decomp_df['deseasonalized'] = decomp_df['original'] - decomp_df['seasonal']

print("계절성 제거 데이터:")
print(decomp_df[['original', 'seasonal', 'deseasonalized']].head(15))

---

## 2.3 Plotly로 시계열 분해 시각화

### 4개 서브플롯으로 분해 결과 시각화

In [ ]:
# 4개 서브플롯 생성
fig = make_subplots(
    rows=4, cols=1,
    subplot_titles=('원본 데이터', '추세 (Trend)', '계절성 (Seasonality)', '잔차 (Residual)'),
    vertical_spacing=0.08
)

# 1. 원본
fig.add_trace(
    go.Scatter(x=decomp_df.index, y=decomp_df['original'], name='원본', line=dict(color='black')),
    row=1, col=1
)

# 2. 추세
fig.add_trace(
    go.Scatter(x=decomp_df.index, y=decomp_df['trend'], name='추세', line=dict(color='blue')),
    row=2, col=1
)

# 3. 계절성
fig.add_trace(
    go.Scatter(x=decomp_df.index, y=decomp_df['seasonal'], name='계절성', line=dict(color='green')),
    row=3, col=1
)

# 4. 잔차
fig.add_trace(
    go.Scatter(x=decomp_df.index, y=decomp_df['residual'], name='잔차', mode='markers', marker=dict(color='red', size=4)),
    row=4, col=1
)

fig.update_layout(
    height=800,
    title_text="시계열 분해 (Additive Model)",
    showlegend=False
)

fig.show()

### 인터랙티브 비교: 원본 vs 계절성 제거

In [ ]:
fig2 = go.Figure()

fig2.add_trace(go.Scatter(
    x=decomp_df.index, y=decomp_df['original'],
    mode='lines+markers', name='원본 데이터',
    line=dict(color='lightblue', width=2)
))

fig2.add_trace(go.Scatter(
    x=decomp_df.index, y=decomp_df['deseasonalized'],
    mode='lines+markers', name='계절성 제거',
    line=dict(color='darkblue', width=2, dash='dash')
))

fig2.add_trace(go.Scatter(
    x=decomp_df.index, y=decomp_df['trend'],
    mode='lines', name='추세선',
    line=dict(color='red', width=3)
))

fig2.update_layout(
    title='원본 vs 계절성 제거 vs 추세',
    xaxis_title='날짜',
    yaxis_title='매출 (만원)',
    hovermode='x unified'
)

fig2.show()

---

## 2.4 이동 통계로 이상치 탐지

### Bollinger Bands 기법

- 이동평균 ± (표준편차 × 2)
- 밴드를 벗어나면 이상치로 간주
- 주가 변동성, 센서 데이터 이상 탐지에 활용

In [ ]:
# 센서 데이터 시뮬레이션 (온도)
np.random.seed(999)
sensor_data = pd.DataFrame({
    'temperature': 25 + np.random.randn(100) * 2
}, index=pd.date_range('2024-01-01', periods=100, freq='H'))

# 일부 이상치 주입
sensor_data.loc[sensor_data.index[20], 'temperature'] = 35  # 이상 고온
sensor_data.loc[sensor_data.index[50], 'temperature'] = 15  # 이상 저온
sensor_data.loc[sensor_data.index[80], 'temperature'] = 40  # 이상 고온

# Bollinger Bands 계산
window = 20
sensor_data['MA'] = sensor_data['temperature'].rolling(window).mean()
sensor_data['STD'] = sensor_data['temperature'].rolling(window).std()
sensor_data['Upper'] = sensor_data['MA'] + 2 * sensor_data['STD']
sensor_data['Lower'] = sensor_data['MA'] - 2 * sensor_data['STD']

# 이상치 플래그
sensor_data['anomaly'] = (
    (sensor_data['temperature'] > sensor_data['Upper']) |
    (sensor_data['temperature'] < sensor_data['Lower'])
)

print("이상치 탐지 결과:")
print(sensor_data[sensor_data['anomaly']])

In [ ]:
# Bollinger Bands 시각화
fig3 = go.Figure()

# 온도 데이터
fig3.add_trace(go.Scatter(
    x=sensor_data.index, y=sensor_data['temperature'],
    mode='lines', name='온도',
    line=dict(color='gray', width=1)
))

# 이동평균
fig3.add_trace(go.Scatter(
    x=sensor_data.index, y=sensor_data['MA'],
    mode='lines', name='이동평균 (20)',
    line=dict(color='blue', width=2)
))

# Upper Band
fig3.add_trace(go.Scatter(
    x=sensor_data.index, y=sensor_data['Upper'],
    mode='lines', name='Upper Band',
    line=dict(color='red', width=1, dash='dash')
))

# Lower Band
fig3.add_trace(go.Scatter(
    x=sensor_data.index, y=sensor_data['Lower'],
    mode='lines', name='Lower Band',
    line=dict(color='red', width=1, dash='dash')
))

# 이상치 마커
anomalies = sensor_data[sensor_data['anomaly']]
fig3.add_trace(go.Scatter(
    x=anomalies.index, y=anomalies['temperature'],
    mode='markers', name='이상치',
    marker=dict(color='orange', size=12, symbol='x')
))

fig3.update_layout(
    title='Bollinger Bands를 활용한 이상치 탐지',
    xaxis_title='시간',
    yaxis_title='온도 (°C)',
    hovermode='x unified'
)

fig3.show()

### 💡 실무 예시: 웹 트래픽 이상 탐지

평소와 다른 급격한 트래픽 증가/감소를 탐지하여 DDoS 공격이나 시스템 장애 파악

In [ ]:
# 웹 트래픽 시뮬레이션 (시간당 요청 수)
np.random.seed(555)
traffic = pd.DataFrame({
    'requests': np.random.poisson(lam=1000, size=200)
}, index=pd.date_range('2024-01-01', periods=200, freq='H'))

# DDoS 공격 시뮬레이션 (갑작스런 트래픽 증가)
traffic.loc[traffic.index[100:105], 'requests'] = np.random.poisson(lam=5000, size=5)

# 이동 평균 및 표준편차
traffic['MA'] = traffic['requests'].rolling(24).mean()
traffic['Upper'] = traffic['MA'] + 3 * traffic['requests'].rolling(24).std()

# 이상치 탐지
traffic['spike'] = traffic['requests'] > traffic['Upper']

print("트래픽 급증 탐지:")
print(traffic[traffic['spike']])

---

## 🎯 실습 퀴즈

**난이도**: ⭐ (쉬움) ~ ⭐⭐⭐⭐⭐ (어려움)

---

### Q1. 일별 매출을 주별로 집계하기 ⭐

**문제**: 다음 일별 매출 데이터를 주별로 다운샘플링하여 합계를 구하세요.

```python
dates = pd.date_range('2024-01-01', periods=14, freq='D')
sales = pd.DataFrame({'revenue': [100, 120, 90, 110, 130, 140, 95, 105, 125, 115, 135, 145, 100, 120]}, index=dates)
```

**기대 결과**: 주별 매출 합계 (2개 주)

In [ ]:
# 여기에 코드를 작성하세요


### Q2. 7일 이동 평균 계산하기 ⭐⭐

**문제**: 주어진 매출 데이터에 7일 이동 평균 컬럼을 추가하세요.

```python
dates = pd.date_range('2024-01-01', periods=30, freq='D')
sales = pd.DataFrame({'revenue': np.random.randint(80, 150, size=30)}, index=dates)
```

**기대 결과**: 'MA_7' 컬럼 추가

In [ ]:
# 여기에 코드를 작성하세요


### Q3. 누적 합계 구하기 ⭐⭐

**문제**: 일별 수익에 대해 연초부터 현재까지 누적 합계를 계산하세요.

```python
dates = pd.date_range('2024-01-01', periods=10, freq='D')
profit = pd.DataFrame({'profit': [50, -20, 30, 40, -10, 60, 20, 30, -15, 45]}, index=dates)
```

**기대 결과**: 'cumulative_profit' 컬럼 추가

In [ ]:
# 여기에 코드를 작성하세요


### Q4. 월별 OHLC 집계 ⭐⭐⭐

**문제**: 일별 주가 데이터를 월별로 다운샘플링하여 OHLC (Open, High, Low, Close)를 계산하세요.

```python
dates = pd.date_range('2024-01-01', '2024-03-31', freq='D')
stock = pd.DataFrame({'close': 50000 + np.cumsum(np.random.randn(len(dates)) * 500)}, index=dates)
```

**기대 결과**: 월별 ['Open', 'High', 'Low', 'Close'] 컬럼

In [ ]:
# 여기에 코드를 작성하세요


### Q5. 업샘플링 후 선형 보간 ⭐⭐⭐

**문제**: 월별 예산 데이터를 일별로 업샘플링한 후 선형 보간(interpolate)하세요.

```python
monthly_budget = pd.DataFrame({'budget': [1000, 1200, 1100]}, index=pd.date_range('2024-01', periods=3, freq='MS'))
```

**기대 결과**: 일별 예산 (선형 보간)

In [ ]:
# 여기에 코드를 작성하세요


### Q6. 이동 표준편차로 변동성 측정 ⭐⭐⭐

**문제**: 주가 데이터에서 20일 이동 표준편차를 계산하여 변동성을 측정하세요.

```python
dates = pd.date_range('2024-01-01', periods=60, freq='D')
stock = pd.DataFrame({'price': 100 + np.cumsum(np.random.randn(60) * 2)}, index=dates)
```

**기대 결과**: 'volatility' 컬럼 (20일 이동 표준편차)

In [ ]:
# 여기에 코드를 작성하세요


### Q7. 지수 가중 이동 평균 (EWM) 계산 ⭐⭐⭐⭐

**문제**: 주가 데이터에서 span=10인 지수 가중 이동 평균을 계산하세요.

```python
dates = pd.date_range('2024-01-01', periods=50, freq='D')
stock = pd.DataFrame({'price': 100 + np.cumsum(np.random.randn(50) * 2)}, index=dates)
```

**기대 결과**: 'EWM_10' 컬럼 추가

In [ ]:
# 여기에 코드를 작성하세요


### Q8. 시계열 분해 수행하기 ⭐⭐⭐⭐

**문제**: 다음 월별 매출 데이터를 Additive 모델로 분해하여 추세, 계절성, 잔차를 추출하세요.

```python
dates = pd.date_range('2022-01', periods=24, freq='MS')
sales = pd.DataFrame({
    'revenue': 100 + np.linspace(0, 50, 24) + 20 * np.sin(np.linspace(0, 4*np.pi, 24)) + np.random.randn(24) * 5
}, index=dates)
```

**힌트**: `from statsmodels.tsa.seasonal import seasonal_decompose`, period=12

**기대 결과**: trend, seasonal, residual 컴포넌트

In [ ]:
# 여기에 코드를 작성하세요


### Q9. 계절성 제거 (Deseasonalize) ⭐⭐⭐⭐

**문제**: Q8의 분해 결과를 사용하여 계절성을 제거한 데이터를 생성하세요.

**기대 결과**: 원본 데이터에서 계절성 컴포넌트를 뺀 값

In [ ]:
# 여기에 코드를 작성하세요


### Q10. Bollinger Bands로 이상치 탐지 ⭐⭐⭐⭐⭐

**문제**: 센서 온도 데이터에서 Bollinger Bands를 계산하고, 밴드를 벗어나는 이상치를 탐지하세요.

```python
np.random.seed(42)
dates = pd.date_range('2024-01-01', periods=100, freq='H')
sensor = pd.DataFrame({'temp': 25 + np.random.randn(100) * 2}, index=dates)
sensor.loc[sensor.index[20], 'temp'] = 35  # 이상치
sensor.loc[sensor.index[80], 'temp'] = 15  # 이상치
```

**요구사항**:
1. 20기간 이동평균 계산
2. Upper/Lower Band 계산 (평균 ± 2 × 표준편차)
3. 이상치 플래그 ('anomaly' 컬럼)
4. Plotly로 시각화 (온도, 밴드, 이상치 마커)

**기대 결과**: 이상치 탐지 완료 + 시각화

In [ ]:
# 여기에 코드를 작성하세요


---

## 📊 학습 정리

### Part 1: 기초 핵심 요약

| 개념 | 핵심 메서드 | 실무 활용 |
|------|------------|----------|
| **resample** | `.resample('W').sum()` | 일별 → 주별/월별 집계, 주기 변환 |
| **다운샘플링** | `.resample('M').agg({...})` | 고빈도 → 저빈도 (데이터 요약) |
| **업샘플링** | `.resample('D').ffill()` / `.interpolate()` | 저빈도 → 고빈도 (결측치 채우기) |
| **rolling** | `.rolling(7).mean()` | 이동평균, 노이즈 제거, 추세 파악 |
| **expanding** | `.expanding().sum()` | 누적 합계, 누적 평균, 연초부터 현재까지 |

### Part 2: 심화 핵심 요약

| 기법 | 문법 | 언제 쓰나? |
|------|------|----------|
| **EWM** | `.ewm(span=10).mean()` | 최근 데이터 가중치 높임 (변동성, 트렌드 추적) |
| **시계열 분해** | `seasonal_decompose(model='additive', period=12)` | 추세/계절성/잔차 분리, 예측 모델링 준비 |
| **계절성 제거** | `원본 - seasonal` | 계절 효과 제거 후 순수 성장 추세 파악 |
| **Bollinger Bands** | `MA ± 2*STD` | 이상치 탐지, 변동성 모니터링 |

### 💡 실무 팁

1. **주기 선택**: 데이터 특성에 맞는 주기 사용 (일별 → 주별은 'W', 월별은 'M')
2. **결측치 처리**: 업샘플링 후 `.ffill()` (계단형) vs `.interpolate()` (선형) 선택
3. **이동평균 윈도우**: 짧을수록 민감, 길수록 평활화 (주가: 5일, 20일, 60일선)
4. **EWM vs SMA**: 최근 변화 빠르게 반영하려면 EWM, 안정적이면 SMA
5. **시계열 분해**: 계절성 있는 데이터(월별, 분기별)는 반드시 분해 후 분석
6. **이상치 탐지**: Bollinger Bands의 2σ 기준은 약 95% 신뢰구간
7. **Plotly 시각화**: `hovermode='x unified'`로 여러 시계열 동시 비교
8. **DatetimeIndex 필수**: resample/rolling 사용 전 `.set_index('date')` 필수